# Estimación del Valor de Diamantes

**Yamil Ernesto Morfa Avalos**

# Introducción

Si bien todos los diamantes están hechos de carbono comprimido, cada diamante individual es único y diferente en sí mismo. Los diamantes tienen en una amplia variedad de formas, tamaños y colores, así como en diversas características internas y externas que se utilizan para su identificación. Todos los diamantes tienen algún valor, pero ¿cómo se determina exactamente cuál diamante es más valioso que otro?

Según https://www.worthy.com/blog/knowledge-center/diamonds/estimate-diamonds-value/

El valor de un diamante se basa en varios factores. Para llegar a un consenso de cómo valorar un diamante, los profesionales de esta industria acordaron un conjunto de pautas particulares que ayudan a establecer su calidad exacta y su atractivo. Estas pautas, o las $4C$, fueron establecidas por el Instituto Gemológico de América y son la base de la estimación del valor de los diamantes. 

Incluyen:

* **Quilate**: Un quilate equivale a 200 miligramos, por lo que este tiene que ver con el peso del diamante.
* **Color**: Un diamante incoloro permite el paso de la luz más fácilmente, lo que resulta en una dispersión mayor de la luz o, en términos sencillos, un diamante más brillante.
* **Claridad**: La descripción de la pureza de un diamante se llama Claridad. Esta se clasifica según la cantidad de imperfecciones internas o marcas externas.
* **Corte**: El corte de un diamante es lo que controla la cantidad de luz reflejada a través del diamante. Es la única caracteristica que depende enteramente de la extracción del mismo.

Nuestro conjunto de datos consta de $53930$ observaciones de $10$ características de diamantes. Como se muestra en la tabla abajo

|Variables|Descripción |Formato|
|:---:|:---:|:---:|
|carat|Peso en Quilates|Numeric|
|cut|Calidad del corte en orden creciente: $Fair$, $Good$, $Very$, $Good$, $Premium$, $Ideal$|String|
|color|Color en orden creciente: $D$ hasta $J$|String|
|clarity|Cuan distinguibles son las inperfecciones internas, en orden decreciente: $FL$, $IF$, $VVS1$, $VVS2$, $VS1$, $VS2$, $SI1$, $SI2$, $I1$, $I2$, $I3$|String|
|depth| La altura del diamante, medida desde la tabla hasta la parte inferior del pabellón, dividida entre el diámetro medio de la corona|Numeric|
|table| El ancho de la tabla del diamante expresado como porcentaje de su diámetro promedio|Numeric|
|price|Precio en USD|Numeric|
|x|largo en mm|Numeric|
|y|ancho en mm|Numeric|
|z|profundidad en mm|Numeric|



# Evaluación de la calidad de los datos.

Primero carguemos los datos y los paquetes necesarios

In [ ]:
library('ggplot2')
library('caret')

In [ ]:
data = read.csv('../input/diamond-valuation-for-gringotts-wizarding-bank/diamonds_data.csv')

# Inspeccionemos los datos
print(length(data$carat))
head(data)

In [ ]:
## Veamos si existen entradas nulas 

print(sum(is.nan(data$carat)))
print(sum(is.nan(data$cut)))
print(sum(is.nan(data$color)))
print(sum(is.nan(data$clarity)))
print(sum(is.nan(data$depth)))
print(sum(is.nan(data$table)))
print(sum(is.nan(data$price)))
print(sum(is.nan(data$x)))
print(sum(is.nan(data$y)))
print(sum(is.nan(data$z)))

No hay entradas nulas en ninguna de las observaciones.

In [ ]:
sum(duplicated(cbind(data$carat, data$cut, data$color, data$clarity, data$depth, 
                     data$table, data$price, data$x, data$y, data$z)))

Arriba se muestra que hay 146 observaciones con exactamente las mismas características. Dado que esto parece un poco improbable, lo consideraremos datos duplicados y se eliminarán. Los datos duplicados pueden afectar los resulados de modelos pues no aportan información diferente.

In [ ]:
data = data[!duplicated(cbind(data$carat, data$cut, data$color, data$clarity, data$depth, 
                     data$table, data$price, data$x, data$y, data$z)),]

In [ ]:
print(length(data$carat))
head(data)

Luego de eliminar los datos duplicados, nos quedamos con un conjunto de $53784$ observaciones.

# Análisis descriptivo y resumen de estadísticas

In [ ]:
summary(data)

In [ ]:
gb<-ggplot(data, aes(color, price, fill=color))
gb+geom_boxplot()

Recordemos que las clases de la variable color están ordenadas desde D-J, donde D es la mejor y J es la peor. De acuerdo con esto podríamos pensar que los precios serían más altos cuanto mejor sea el color. Sin embargo, en el gráfico anterior podemos ver que esta relación no existe, porque para el color D, por ejemplo, tenemos un precio medio más bajo que para el color J.

También tenemos datos que podemos considerarlos atípicos ya que están por encima del percentil 75. 

Este comportamiento atípico posiblemente se debe a que es necesario tener en cuenta otras variables.

In [ ]:
gb<-ggplot(data, aes(clarity, price, fill=clarity))
gb+geom_boxplot()

gb<-ggplot(data, aes(cut, price, fill=cut))
gb+geom_boxplot()

Lo mismo ocurre si consideramos la variable claridad (en orden de mejor a peor: FL, IF, VVS1, VVS2, VS1, VS2, SI1, SI2, I1, I2, I3) donde la clase SI2 tiene el valor de precio promedio más alto, no siendo la clase de mejor calidad. Como se muestra arriba, lo mismo ocurre con la variable corte. Por tanto, no parece factible realizar un análisis de precios con ninguna de estas variables por separado.

In [ ]:
ggplot(data, aes(carat, price)) + geom_point()

In [ ]:
cat('Correlación price vs carat: ', cor(data$price, data$carat), '\n')

En el gráfico anterior podemos ver que para la variable 'carat' se puede observar una correlación positiva con el precio.

Veamos que pasa con el resto de variables.

In [ ]:
ggplot(data, aes(depth, price)) + geom_point()
ggplot(data, aes(table, price)) + geom_point()
ggplot(data, aes(x, price)) + geom_point()
ggplot(data, aes(y, price)) + geom_point()
ggplot(data, aes(z, price)) + geom_point()

In [ ]:
cat('Correlación price vs depth: ', cor(data$depth, data$price), '\n')
cat('Correlación price vs table: ', cor(data$price, data$table), '\n')
cat('Correlación price vs x: ', cor(data$price, data$x), '\n')
cat('Correlación price vs y: ', cor(data$price, data$y), '\n')
cat('Correlación price vs z: ', cor(data$price, data$z), '\n')

Como podemos ver en las gráficas de arriba la variable price parece no depender de las variables 'depth', 'table', ya que no se observa una fuerte correlación (una correlación positiva fuerte, se considera usualmente mayor que $0.5$). Así, podemos pensar en eliminar estas variables para simplificar nuestra tarea.

In [ ]:
data = data[,-5]
data = data[,-5]

In [ ]:
head(data,10)

Como vimos anteriormente, el precio tiende a aumentar a medida que aumenta el valor en quilates.

In [ ]:
multiplot <- function(..., plotlist=NULL, file, cols=1, layout=NULL) {
  library(grid)

  # Make a list from the ... arguments and plotlist
  plots <- c(list(...), plotlist)

  numPlots = length(plots)

  # If layout is NULL, then use 'cols' to determine layout
  if (is.null(layout)) {
    # Make the panel
    # ncol: Number of columns of plots
    # nrow: Number of rows needed, calculated from # of cols
    layout <- matrix(seq(1, cols * ceiling(numPlots/cols)),
                    ncol = cols, nrow = ceiling(numPlots/cols))
  }

 if (numPlots==1) {
    print(plots[[1]])

  } else {
    # Set up the page
    grid.newpage()
    pushViewport(viewport(layout = grid.layout(nrow(layout), ncol(layout))))

    # Make each plot, in the correct location
    for (i in 1:numPlots) {
      # Get the i,j matrix positions of the regions that contain this subplot
      matchidx <- as.data.frame(which(layout == i, arr.ind = TRUE))

      print(plots[[i]], vp = viewport(layout.pos.row = matchidx$row,
                                      layout.pos.col = matchidx$col))
    }
  }
}

In [ ]:
gp = ggplot(data, aes(carat, price))
plot1 = gp+geom_point(aes(colour = color))
plot2 = gp+geom_point(aes(colour = clarity))
plot3 = gp+geom_point(aes(colour = cut))

multiplot(plot1,plot2,plot3, cols=1)
rm(plot1, plot2, plot3)

En el gráfico anterior tenemos los puntos en quilates contra el precio de cada diamante en la base de datos, los colores se asignan según las categorías de las variables: 'color', 'clarity' o 'cut', respectivamente. Podemos ver que categorizando los precios por 'quilates' usando 'color' y 'clarity' como etiqueta, tenemos bandas de color bien definidas. Entonces es factible pensar que considerando alguna de estas variables, o ambas, en conjunto con 'quilate' podemos estudiar el precio. Sin embargo, las características de 'cut' parece no poder distinguir entre precios a través de la variable de quilates.

In [ ]:
gp+geom_point(aes(colour = cut))+facet_grid(rows = vars(cut))

Como podemos ver en el gráfico, los puntos relacionados con cortes 'good', 'ideal', 'premium' y 'very good' están distribuidos casi por igual

Veremos a continuación que ocurre lo mismo con 'x' contra el 'precio'

In [ ]:
gp<-ggplot(data, aes(x, price))
gp+geom_point()
plot1 = gp+geom_point(aes(colour = color))
plot2 = gp+geom_point(aes(colour = clarity))
plot3 = gp+geom_point(aes(colour = cut))

multiplot(plot1,plot2,plot3, cols=1)
rm(plot1, plot2, plot3)

gp+geom_point(aes(colour = cut))+facet_grid(rows = vars(cut))

Aparentemente el comportamiento es muy similar al que vimos anteriormente en el gráfico de 'carat' contra 'price'. Esto nos hace preguntarnos si existe una relación entre 'x' y 'quilate'

In [ ]:
gp<-ggplot(data, aes(x, carat)) 
gp+geom_point()+ stat_smooth(method = lm, formula = y ~ poly(x, 2))

Vemos que existe una relación clara entre 'x' y 'carat'. Eliminemos los valores atípicos y creemos un modelo polinomial para confirmar

In [ ]:
data = data[data$x>3, ]

In [ ]:
gp<-ggplot(data, aes(x, carat)) 
gp+geom_point()+ stat_smooth(method = lm, formula = y ~ poly(x, 2))

In [ ]:
polyMod = train(carat ~ poly(x, 2) , data=data, method='lm')
print(polyMod)
summary(polyMod)

Parece que nuestras variables 'x' y 'carat' están relacionadas por un polinomio de grado 2 de la forma:
$$ carat (x) = c_1x ^ {2} + c_2x + c_3 $$
los coeficientes estimados (Estimate) tienen varianzas muy pequeñas (Std. Error) y el ajuste a los datos (Rsquared) también es pequeño. Dado que existe una relación tan estrecha entre estas variables, es posible eliminar una de ellas. Esto porque en un modelo de regresión  siempre se puede poner una en función de la otra.

In [ ]:
data = data[,-6]


# Hipótesis y modelado

Para el modelado, consideramos 'variables independientes' o 'características' y la 'variable dependiente' o 'target', en este caso 'price' como un indicador del valor de un diamante. Con un modelo, intentamos predecir el precio a partir de las 'características' y su relación. Entonces, luego de haber realizado un análisis de los datos, procedemos a realizar un modelo donde se pueden predecir los valores de precio a partir de las variables.

Dado que disponemos de datos con su respectivo precio podemos utilizar el aprendizaje supervisado. Comencemos con la preparación de los datos, para esto, cada dato del conjunto de entrenamiento se representa como una matriz que llamaremos vector de características. Como resultado, tendremos una matriz de datos donde cada fila representa una observación.

In [ ]:
X = data
X$cut = as.factor(data$cut)
X$color = as.factor(data$color)
X$clarity = as.factor(data$clarity)

#rm(data)
head(X)

Como vimos en la sección anterior, aparentemente, es mucho más fácil identificar un patrón determinado si agrupamos los datos que representan más de una variable. Por ejemplo: Si tomamos los datos que tienen como color la clase $J$ y como tipo de corte, la clase $Premium$ obtenemos lo que se muestra en la siguiente grafica.

In [ ]:
X$ccc = rep(NA, dim(X)[1])
bool1 = X$color == 'J' 
bool2 = X$cut == 'Premium'
bool = c()
for (i in 1:length(bool1)){
    bool[i] = bool1[i] && bool2[i]
}
gp<-ggplot(X[bool,], aes(carat, price))
gp+geom_point()+ stat_smooth(method = lm, formula = y ~ poly(x, 3))

Otro ejemplo, considerando todas las variables categóricas:

In [ ]:
bool1 = X$color == 'J' 
bool2 = X$cut == 'Premium'
bool3 = X$clarity == 'SI1'
bool = c()
for (i in 1:length(bool1)){
    bool[i] = bool1[i] && bool2[i] && bool3[i]
}
gp<-ggplot(X[bool,], aes(carat, price))
gp+geom_point()+ stat_smooth(method = lm, formula = y ~ poly(x, 3))

Siguiendo esta idea, consideremos las variables 'color', 'claridad' y 'corte' como una sola variable, asignando a cada combinación posible de estas un valor diferente. Llamaremos a esta nueva variable 'ccc' por las iniciales de las variables combinadas

In [ ]:
X$ccc = rep(NA, dim(X)[1])

colors = sort(unique(X$color))
cuts = sort(unique(X$cut))
claritys = sort(unique(X$clarity))

transformation = matrix(NA, nrow = length(colors)*length(cuts)*length(claritys), ncol = 4)
c = 0
for (col in colors){
    bool1 <- X$color == col 
    for (cu in cuts){
        bool2 <- X$cut == cu
        for (cl in claritys){
            bool3 <- X$clarity == cl
            bool = c()
            for (i in 1:length(bool1)){
                bool[i] = bool1[i] && bool2[i] && bool3[i]
            }
            c = c + 1
            transformation[c, ] = c(cu, col, cl, c)
            if (sum(bool)!=0){
                X[bool,]$ccc = c
            }
        }
    }
}

head(X)

In [ ]:
head(transformation)

In [ ]:
gp<-ggplot(X, aes(carat, price))
gp+geom_point(aes(colour = ccc))

El supuesto anterior parece tener sentido debido a la forma en que se comporta la distribución de los colores representados por la nueva variable 'ccc'. 

## Selección de modelo

Existen varios modelos para predecir un valor numérico. El análisis de regresión es una técnica de modelado predictivo que analiza la relación entre la variable target o dependiente y la variable independiente en un conjunto de datos. Los diferentes tipos de técnicas de análisis de regresión se utilizan cuando las variables objetivo e independientes muestran una relación lineal o no lineal entre sí, y la variable objetivo contiene valores continuos.

Para probar, antes de hacer predicciones sobre datos desconocidos, es necesario tener datos de prueba cuyas predicciones reales sean conocidas.

In [ ]:
sample = sample(2, dim(X)[1], replace = TRUE, prob = c(0.8,0.2))
train = X[sample==1,]
test = X[sample==2, ]

In [ ]:
cat('Dimensión de conjunto de entreneamiento: ',  dim(train), '\n')
cat('Dimensión de conjunto de prueba o validación: ',  dim(test), '\n')

Antes de realizar predicciones con un modelo, necesitamos una medida para determinar cuan preciso es nuestro modelo. En clasificación usualmente usamos la accuracy que es la cantidad de datos bien clasificados entre la cantidad total de datos clasificados, pero en un modelo de regresión ¿cómo determinar si nuestra prediccion es correcta? 

Una forma común en el análisis de regresión es usar el Error cuadrático medio $MSE$
$$ MSE = \frac{1}{n}\sum_{i = 1}^{n} \left( f\left(X\right) - y \right)^{2} $$
donde $f\left(X\right)$ es el valor predicho por un modelo $f$ y $y$ es el valor real. Sin embargo este es muy sensible a los outlayes, una alternativa es $MAE$ o error absoluto medio:
$$ MAE = \frac{1}{n}\sum_{i = 1}^{n} \left| f\left(X\right) - y \right|$$


In [ ]:
MSE = function(y_pred, y_true){
    return(mean((y_pred-y_true)^2))
}
MAE = function(y_pred, y_true){
    return(mean(abs(y_pred-y_true)))
}
R2

In [ ]:
head(train)

## K-nn

Knn es un modelos que usualmente se utiliza para clasificación, el algoritmo es sencillo, solamanete su utiliza una función de similitud para identificar los $k$ vecinos más cercanos ($k$ - nearest neighbors) y se le asigna a la observación nueva la clase más repetida en el conjunto de vecinos. Este puede ser modificado para regresión adjudicando a la nueva observación la media del conjunto de vecinos más cercanos.

In [ ]:
knnMod = train(price ~ carat+ccc+y+z, data=train, method='knn')
print(knnMod)
summary(knnMod)

In [ ]:
varimp <- varImp(knnMod)
plot(varimp, main="Variable Importance")

## Prdict
predicted <- predict(knnMod, test)

In [ ]:
cat('MSE para K-nn: ', MSE(predicted, test$price), '\n')
cat('MAE para K-nn: ', MAE(predicted, test$price), '\n')

Como podemos ver, el modelo no le da mucho valor a la variable 'ccc'. Esto sugiere que probemos descartando la variable 'ccc' y volviendo a las variables originales.

In [ ]:
train$clarity = unclass(train$clarity)
train$color = unclass(train$color)
train$cut = unclass(train$cut)

test$clarity = unclass(test$clarity)
test$color = unclass(test$color)
test$cut = unclass(test$cut)

In [ ]:
knnMod1 = train(price ~ carat+clarity+color+cut+y+z, data=train, method='knn')
print(knnMod1)
summary(knnMod1)

In [ ]:
varimp <- varImp(knnMod1)
plot(varimp, main="Variable Importance")

## Prdict
predicted <- predict(knnMod, test)
cat('MSE para K-nn: ', MSE(predicted, test$price), '\n')
cat('MAE para K-nn: ', MAE(predicted, test$price), '\n')

Como se puede ver, se comporta muy similar al anterior, aunque hay una mejora en MAE. El MAE como vimos anteriormente, describe el error absoluto medio que se comete en la predicción. Dado los estadísticos de la variable precio, con una media de $3933$ y un máximo de $18823$, un error de $331$ es aceptable. En adelante consideraremos como un error aceptable al valor que esté por debajo del primer cuantil $MAE<951$.

## CART 

Los árboles de regresión y clasificación son una herramienta útil para porblemas que presentan comportamientos no lineales. La idea consiste en particionar el conjunto de entrenamiento mediante reglas determinadas, hasta obtener nodos terminales que devuelven, una clase en caso de clasificación o un valor en caso de regresión. Cada observación nueva pasa por el árbol y termina en único nodo.

In [ ]:
CARTMod = train(price ~ carat + ccc+y+z, data=train, method='rpart2')
CARTMod1 = train(price ~ carat+clarity+color+cut+y+z, data=train, method='rpart2')

In [ ]:
varimp <- varImp(CARTMod)
varimp1 <- varImp(CARTMod1)
plot(varimp, main="Variable Importance: Modelo con ccc")
plot(varimp1, main="Variable Importance: MOdelo sin ccc")

## Prdict
predicted <- predict(CARTMod, test)
predicted1 <- predict(CARTMod1, test)
cat('MSE para CART: Modelo con ccc: ', MSE(predicted, test$price), '\n')
cat('MAE para CART: Modelo con ccc', MAE(predicted, test$price), '\n')
cat('MSE para CART: Modelo sin ccc: ', MSE(predicted1, test$price), '\n')
cat('MAE para CART: Modelo sin ccc', MAE(predicted1, test$price), '\n')

Podemos ver que aunque el modelo CART no supone una mejora con respecto a $KNN$, sus valores de $MAE$ están por debajo del humbral supuesto, al menos para el caso en que no se considera la variable 'ccc'

In [ ]:
nnMod = train(price ~ carat+ccc+y+z, data=train, method='nnet', trace = FALSE)
nnMod1 = train(price ~ carat+clarity+color+cut+y+z, data=train, method='nnet', trace = FALSE)


In [ ]:
print(nnMod)
summary(nnMod)
print(nnMod1)
summary(nnMod1)

varimp <- varImp(nnMod)
varimp1 <- varImp(nnMod1)
plot(varimp, main="Variable Importance: Modelo con ccc")
plot(varimp1, main="Variable Importance: Modelo sin ccc")

## Prdict
predicted <- predict(nnMod, test)
predicted1 <- predict(nnMod1, test)
cat('MSE para NNet: Modelo con ccc: ', MSE(predicted, test$price), '\n')
cat('MSE para NNet: Modelo sin ccc: ', MSE(predicted1, test$price), '\n')
cat('MAE para NNet: Modelo con ccc: ', MAE(predicted, test$price), '\n')
cat('MAE para NNet: Modelo sin ccc: ', MAE(predicted1, test$price), '\n')

El modelo NNet no tuvo, en ninguno de los casos un buen rendimiento. De los modelos considerados los que parecen funcionar mejor son $Knn$ y $CART$. A continuación realizamos una comparación entre estos.

In [ ]:
results <- resamples(list(KNN=knnMod, KNN1 = knnMod1, CART = CARTMod, CART1=CARTMod1))

In [ ]:
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)

Esta comparación realiza 25 muestreos de los datos de entrenamiento y calcula las métricas que se muestran. De estas se muestran los estadísticos mínimo, primer cuantil, moda, media, tercer cuantil y máximo. Como podemos ver el modelo $KNN$, usando las variables color, clarity y cut tiene los valores más bajos. Visto esto, de los modelos tratados, el que mejor se comporta a la hora de predecir los valores de los diamantes es el $KNN$.

# Conclusión y próximos pasos

Realizamos una limpieza de los datos, una descripción estadística y un modelado para el problema, el cual finalmente fue validado. Los métodos estudiados para la clasificación de los diamantes fueron, K-nn, CART y NNet. Además, para distinguir el mejor de estos método se utilizaron distintas métricas: error cuadrático medio, error absoluto medio y R-squared. Pudimos observar que los mejores resultados se obtuvieron con el método K-nn ya que mostraron los menores valores de error utilizando las distintas métricas. Finalmente tenemos un modelo para poder distinguir el valor de los diamantes en cuanto a a su precio mediante las características que estos presentan.